In [1]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('../data/Raptor Contract.docx.pdf')
pages = loader.load_and_split()

In [2]:
len(pages)

76

In [3]:
length = 0
count = 0
for page in pages:
    length += len(page.page_content)
    count += 1

length, count

(225087, 76)

In [4]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter

CHUNK_SIZE = 300
CHUNK_OVERLAP = 30

text_splitter = CharacterTextSplitter(chunk_size = CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
docs = text_splitter.split_documents(pages)

In [5]:
print(len(docs[0].page_content))

801


In [9]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Qdrant

embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')

qdrant = Qdrant.from_documents(
    docs, 
    embeddings,
    path='local_qdrant',
    collection_name="contract_documents",
    force_recreate=True,
)

In [10]:
query = 'What is the business objective of the project?'

found_docs = qdrant.similarity_search(query, k=2)

print(found_docs[0].page_content)

of
such
Person
to
secure
(regardless
of
whether
assumed
by
such
Person),
the
obligations 
described
in
clauses
(a)
through
(d)
above
of
any
other
Person
that
is
not
an
Acquired
Company .
“
Employee
Plan
”
means
any
plan,
program,
policy ,
agreement,
arrangement,
practice, 
workers’
benefits
or
Contractual
Obligation,
whether
or
not
reduced
to
writing,
whether
or
not 
subject
to
ERISA,
and
whether
covering
a
single
individual
or
a
group
of
individuals,
that
is
(a) a 
welfare
plan
within
the
meaning
of
Section
3(1)
of
ERISA,
(b) a
pension
benefit
plan
within
the 
meaning
of
Section
3(2)
of
ERISA
or
a
pension
arrangement,
disability
benefit
and
any
other 
provident
fund
(including
study
fund),
(c) a
vacation
benefit,
a
stock
bonus,
stock
purchase, 
stock
option,
restricted
stock,
stock
appreciation
right
or
similar
equity-based
plan,
program, 
policy ,
agreement,
arrangement,
practice,
workers’
benefits
or
Contractual
Obligation,
including 
without
limitation
the
Company
Stock
Plans,
or
(

In [11]:
print(len(found_docs[0].page_content))

3904


### Generation

### load_qa_chain


In [12]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAIChat

chain = load_qa_chain(OpenAIChat(temperature=0), chain_type='stuff')

answer = chain.run(input_documents=found_docs, question=query)

/home/misge/miniconda3/envs/wk-11/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAIChat` was deprecated in langchain-community 0.0.1 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/home/misge/miniconda3/envs/wk-11/lib/python3.12/site-packages/langchain_community/llms/openai.py:1070: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
/home/misge/miniconda3/envs/wk-11/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. U

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [13]:
answer

NameError: name 'answer' is not defined

#### load_qa_with_sources_chain

In [14]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

chain = load_qa_with_sources_chain(OpenAIChat(temperature=0.2, model_name='gpt-3.5-turbo'), chain_type='stuff')

answer = chain({'input_documents': found_docs, 'question': query}, return_only_outputs=True)

/home/misge/miniconda3/envs/wk-11/lib/python3.12/site-packages/langchain_community/llms/openai.py:1070: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
/home/misge/miniconda3/envs/wk-11/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [15]:
answer 

NameError: name 'answer' is not defined

#### RetrieverQA

In [16]:
from langchain.chains import RetrievalQA 
from langchain.chains import RetrievalQAWithSourcesChain 

qa = RetrievalQA.from_chain_type(llm=OpenAIChat(temperature=0.2, model_name='gpt-3.5-turbo'), chain_type='stuff', retriever=qdrant.as_retriever())

answer = qa.run(query)

/home/misge/miniconda3/envs/wk-11/lib/python3.12/site-packages/langchain_community/llms/openai.py:1070: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
answer 

#### RetrieverQAWithSourcesChain

In [17]:
chain = RetrievalQAWithSourcesChain.from_chain_type(OpenAIChat(temperature=0.2, model_name='gpt-3.5-turbo'), chain_type='stuff', retriever=qdrant.as_retriever())

answer = chain({'question': query}, return_only_outputs=True)


/home/misge/miniconda3/envs/wk-11/lib/python3.12/site-packages/langchain_community/llms/openai.py:1070: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
ans